# Tensile test example

### Data conversion from ELN to RDF for a tensile test experiment sample

This notebook uses an example of the data which is synthetic, but that fits the format of the data derived from the experiments. The goal is to see how the scripts convert the data from the compressed format to an RDF graph and turtle file. 
Let's start by importing our functions and needed libraries. 

In [8]:
#import libraries
import yaml
from rdflib import Graph
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the main directory
main_dir_path = os.path.join(current_dir, '..')

# Add the path to the Python path
sys.path.append(main_dir_path)

# import our functions from src
from src import *

Let's define the path to the eln export, provide a name for the output file, set the path of the keymap or configuration file, the pattern for the json file and also define a boolean variable that reflect our choice of whether we want to have the plot of the graph of the output or not.

In [6]:
# paths for the data
eln_export = 'tensile_test_data_example.eln'
output_file = 'tensile_test.ttl'
keymap_file_path = '../keymaps/tensile_test.yml'
pattern = 'ftw.json'
plot = True  # whether or not to plot the RDF graph

First, we create the graph and bind the prefixes to it. Then, we extract, parse and process the data with the mapping defined in the keymap file. In the end, the graph is serialized to Turtle format and the plot is also given as an output.  

In [5]:
# Load the keymap
with open(keymap_file_path, 'r') as f:
    keymap = yaml.safe_load(f)

# Create a fresh RDF graph
data_graph = Graph()

# Bind the prefixes to the RDF graph
bind_prefixes_to_graph(data_graph, keymap['namespaces'])

# Process each file in the ELN export
for file_name, json_data in json_data_from_zip_generator(eln_export, pattern):
        elab_data = parse_json_export(json_data[0])
        process_data_with_mapping(data_graph, elab_data, keymap)

# Serialize the graph to a Turtle file
data_graph.serialize(destination=output_file, format='turtle')

# Plot RDF graph if plot is set to True
if plot:
    image_filename = os.path.splitext(output_file)[0] + '.png'
    plot_rdf_graph(data_graph, image_filename)